In [1]:
from bs4 import BeautifulSoup
import time
from random import randint
import pandas as pd
import re
import sqlite3
import numpy as np
import spacy

In [2]:
conn = sqlite3.connect('lyrics.db')

In [3]:
nlp = spacy.load('en')

## Clean Lyrics Database

In [4]:
records = pd.read_sql_query("select * from lyrics_abc where valid = 1;",conn)

In [35]:
# allrecords = pd.read_sql_query("select * from lyrics_abc;",conn)
# allrecords['duplicate'] = allrecords['url'].duplicated()

# def update(x):
#     if(x['duplicate']==True)&(x['status']==0):
#         return 1
#     elif x['status'] == 1 : 
#         return 1
#     else : return 0

# allrecords['new_status'] = allrecords.apply(lambda x: update(x),axis=1)
# allrecords['status'] = allrecords['new_status']
# del allrecords['new_status']

# allrecords[allrecords['url']=='http://cyberhymnal.org/htm/b/e/beautxms.htm']
# print('scrapped:',allrecords[allrecords['status']==1]['status'].count())
# print('not scrapped:',allrecords[allrecords['status']==0]['status'].count())
# print('total:',allrecords['status'].count())
# del allrecords['duplicate']

# allrecords.to_sql('lyrics_abc',conn,if_exists='replace')

In [72]:
# def update(x):
#     if x['fullhtml'] is None:
#         return 0
#     elif x['song_name'] == "  Page not found - CyberHymnal" :
#         return 0
#     else :
#         return 1

# records[records['fullhtml'].notnull()]
# records.iloc[7692]['song_name']
# records['valid'] = records.apply(lambda x:update(x),axis=1)
# records.to_sql('lyrics_abc',conn,if_exists='replace')

In [75]:
columns = ['key','title','composer_music','music_year','composer_lyrics','lyrics_year','lyrics']
df = pd.DataFrame(columns=columns)

for reckey,record in records.iterrows():
    soup = BeautifulSoup(record['fullhtml'],"lxml")
    key = reckey
    title = soup.title.get_text()
    #print(title," ",key)
    
    composer = re.findall("Words:(.*) Music:(.*?)\"",record['meta'])
    if len(composer) > 0 :
        music = re.findall("[ ]?(.*?)[,.].*",composer[0][1])
        #music = re.findall("(.*)[,.].*",composer[0][1])
        music_yr = re.findall(".*(\d{4})",composer[0][1])
        composer_music = music[0] if len(music) > 0 else 0
        music_year = music_yr[0] if len(music_yr) > 0 else 0
        #print("Music:",composer_music)
        #print("Year:",music_year)


        words = re.findall("[ ]?(.*?),.*",composer[0][0])
        words_year = re.findall(".*(\d{4})",composer[0][0])   

        composer_lyrics = words[0] if len(words) > 0 else 0
        lyrics_year = words_year[0] if len(words_year) > 0 else 0
        #print("Lyrics:",composer_lyrics)
        #print("Year:",lyrics_year)
    else : 
        composer_lyrics = ""
        lyrics_year = 0 
        composer_music = ""
        music_year = 0 
          
    lyrics = BeautifulSoup(record['lyrics'],"lxml").get_text()
    #print(lyrics)
    
    row = [key,title,composer_music,music_year,composer_lyrics,lyrics_year,lyrics]
    #print(composer)
    #print([composer_music,music_year,composer_lyrics,lyrics_year])
    df.loc[len(df)] = row 
    
    
    #print('--------------------------')
df.to_sql('cleaned_lyrics',conn,if_exists='replace')

## Analyze Database

In [5]:
records = pd.read_sql_query("select * from cleaned_lyrics;",conn)

In [21]:
lyrics = records['lyrics'][0]
print(lyrics)
clean_lyrics = lyrics.replace('\r\n',' ').replace('\n',' ')
print(clean_lyrics)


“Abba, Father!” We approach Thee
In our Savior’s precious Name;
We, Thy children, here assembled,
Now Thy promised blessing claim;
From our sins His blood hath washed us,
’Tis through Him our souls draw nigh,
And Thy Spirit, too, hath taught us,
“Abba, Father,” thus to cry.
Once as prodigals we wandered
In our folly far from Thee,
But Thy grace, o’er sin abounding,
Rescued us from misery;
Thou Thy prodigals hast pardoned,
Kissed us with a Father’s love,
Spread the festive board, and called us,
E’er to dwell with Thee above.
Clothed in garments of salvation,
At Thy table is our place,
We rejoice, and Thou rejoicest,
In the riches of Thy grace;
“It is meet,” we hear Thee saying,
“We should merry and be glad,
I have found My once lost children,
Now they live who once were dead.”
“Abba, Father!” all adore Thee,
All rejoice in Heav’n above,
While in us they learn the wonders
Of Thy wisdom, grace, and love;
Soon before Thy throne assembled,
All Thy children shall proclaim,
“Glory, everlasti

In [22]:
slyrics = nlp(lyrics)
sclyrics = nlp(clean_lyrics)

swords = [token.text for token in slyrics if token.is_stop != True and token.is_punct != True]
scwords = [token.text for token in sclyrics if token.is_stop != True and token.is_punct != True]
# words = [token.text for token in lyrics_1 if token.is_stop != True and token.is_punc != True and token.pos_ == 'NOUN']
print(swords)
print("------------------")
print(scwords)

# for num,sentence in enumerate(lyrics_1.sents):
#     print(num," ",sentence)


['\n', 'Abba', 'Father', 'approach', 'Thee', '\r\n', 'Savior', '’s', 'precious', '\r\n', 'Thy', 'children', 'assembled', '\r\n', 'Thy', 'promised', 'blessing', 'claim', '\r\n', 'sins', 'blood', 'hath', 'washed', '\r\n', '’Tis', 'souls', 'draw', 'nigh', '\r\n', 'Thy', 'Spirit', 'hath', 'taught', '\r\n', 'Abba', 'Father', 'cry', '\n', 'prodigals', 'wandered', '\r\n', 'folly', 'far', 'Thee', '\r\n', 'Thy', 'grace', 'o’er', 'sin', 'abounding', '\r\n', 'Rescued', 'misery', '\r\n', 'Thou', 'Thy', 'prodigals', 'hast', 'pardoned', '\r\n', 'Kissed', 'Father', '’s', 'love', '\r\n', 'Spread', 'festive', 'board', 'called', '\r\n', 'E’er', 'dwell', 'Thee', '\n', 'Clothed', 'garments', 'salvation', '\r\n', 'Thy', 'table', 'place', '\r\n', 'rejoice', 'Thou', 'rejoicest', '\r\n', 'riches', 'Thy', 'grace', '\r\n', 'meet', 'hear', 'Thee', 'saying', '\r\n', 'merry', 'glad', '\r\n', 'found', 'lost', 'children', '\r\n', 'live', 'dead', '\n', 'Abba', 'Father', 'adore', 'Thee', '\r\n', 'rejoice', 'Heav’n', '

In [ ]:
## Clean the Reviews.

In [ ]:
## Perform LSTM 
# https://nicschrading.com/project/Intro-to-NLP-with-spaCy/
# http://blog.sharepointexperience.com/2016/01/nlp-and-sharepoint-part-1/
# https://github.com/JonathanReeve/advanced-text-analysis-workshop-2017/blob/master/advanced-text-analysis.ipynb
# http://nbviewer.jupyter.org/github/skipgram/modern-nlp-in-python/blob/master/executable/Modern_NLP_in_Python.ipynb